# Lab session 1- QML- QAIOT (Quantum Kernel Method)

Let's dive to our very first quantum-inspired SVM

This cell appears to be setting up the environment and importing necessary modules for a machine learning task involving classical and potentially quantum component

In [1]:
import numpy as np

from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers


np.random.seed(42)

Overall, Following code cell prepares the data by loading the Iris dataset, selecting a subset of samples, standardizing the features, and splitting the data into training and testing sets for further processing.

In [2]:
X, y = load_iris(return_X_y=True)

# pick inputs and labels from the first two classes only,
# corresponding to the first 100 samples
X = X[:100]
y = y[:100]

# scaling the inputs is important since the embedding we use is periodic
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# scaling the labels to -1, 1 is important for the SVM and the
# definition of a hinge loss
y_scaled = 2 * (y - 0.5)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled)

Here, len(X_train[0]) returns the length (number of elements) of the first sample in the training data X_train. Since each sample represents a set of features, the length of the first sample gives the number of features. In the context of quantum machine learning, this number is often used to determine the number of qubits required to encode the features into a quantum state.

In [3]:
n_qubits = len(X_train[0])
n_qubits

4

In [4]:
dev_kernel = qml.device("default.qubit", wires=n_qubits)

projector = np.zeros((2**n_qubits, 2**n_qubits))
projector[0, 0] = 1

@qml.qnode(dev_kernel, interface="autograd")
def kernel(x1, x2):
    """The quantum kernel."""
    AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

In [5]:
kernel(X_train[0], X_train[0])

tensor(1., requires_grad=True)

Upcoming line calculates the kernel matrix. It uses a nested list comprehension to iterate over every pair of elements a from set A and b from set B. For each pair, it evaluates the kernel function defined earlier with a and b as arguments to get the kernel value, and these values are used to populate the matrix.

The resulting matrix is returned as a NumPy array, where each entry is the value of the quantum kernel function evaluated on the corresponding pairs of elements from sets A and B.

In [6]:
def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel(a, b) for b in B] for a in A])

In [7]:
svm = SVC(kernel=kernel_matrix).fit(X_train, y_train)

In [8]:
predictions = svm.predict(X_test)
accuracy_score(predictions, y_test)

1.0